# Simplest simulation of confidence reduction

- Two estimators
- Each one has a probability distribution
- They exchange their estimates
    - by the naive
    - by the conservative data exchange
- What if they repeated the exchange multiple times?
- What if the exchanged data gains some noises?

